# 🤖 NL2SQL Agent - Simple Tool-Based Approach

A clean, simple implementation of a Natural Language to SQL agent using Azure OpenAI with function calling.

**Key Features:**
- Single intelligent agent orchestrates the entire process
- Tool-based architecture for modularity
- Dataset agnostic - works with any SQLite database
- Minimal complexity for customer demonstration

## 📦 Setup and Dependencies

In [14]:
# Core dependencies
import sqlite3
import json
import os
from typing import Dict, List, Any
from openai import AzureOpenAI

# Azure OpenAI Configuration
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_MODEL = os.getenv("AZURE_OPENAI_MODEL")

## 🛠️ Tool Functions

Simple, focused tools that the agent can use:

In [15]:
def discover_database_schema(database_path: str) -> str:
    """
    Tool: Discover database schema (tables and columns)
    """
    try:
        conn = sqlite3.connect(database_path)
        cursor = conn.cursor()
        
        # Get all tables
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%'")
        tables = cursor.fetchall()
        
        schema_info = {}
        
        for (table_name,) in tables:
            # Get columns for each table
            cursor.execute(f"PRAGMA table_info({table_name})")
            columns = cursor.fetchall()
            
            schema_info[table_name] = [
                {"name": col[1], "type": col[2], "nullable": not col[3]}
                for col in columns
            ]
        
        conn.close()
        return json.dumps(schema_info, indent=2)
        
    except Exception as e:
        return f"Error discovering schema: {str(e)}"


def generate_sql_query(question: str, schema_info: str) -> str:
    """
    Tool: Generate SQL query from natural language question
    """
    try:
        client = AzureOpenAI(
            azure_endpoint=AZURE_OPENAI_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
            api_version=AZURE_OPENAI_API_VERSION
        )
        
        prompt = f"""
You are a SQL expert. Generate a SQLite query for this question.

Database Schema:
{schema_info}

Question: {question}

IMPORTANT GUIDELINES:
1. When asked "how many" or for counts, provide a query that returns the actual rows that match the criteria, NOT just COUNT(*).
2. This allows for validation of the results by seeing the actual data.
3. If you must provide a count, also include key identifying columns so we can verify the results.
4. For aggregate questions (average, sum, etc.), include the underlying rows when possible.
5. Use SELECT * or SELECT key_columns instead of SELECT COUNT(*) when feasible.

Return only the SQL query - no explanations or formatting.
"""
        
        response = client.chat.completions.create(
            model=AZURE_OPENAI_MODEL,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            max_tokens=300
        )
        
        return response.choices[0].message.content.strip()
        
    except Exception as e:
        return f"Error generating SQL: {str(e)}"


def execute_sql_query(sql_query: str, database_path: str) -> str:
    """
    Tool: Execute SQL query and return results
    """
    try:
        # Basic safety check
        if not sql_query.strip().upper().startswith('SELECT'):
            return "Error: Only SELECT queries are allowed"
            
        conn = sqlite3.connect(database_path)
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        cursor.execute(sql_query)
        rows = cursor.fetchall()
        
        # Convert to list of dictionaries
        results = [dict(row) for row in rows]
        
        conn.close()
        return json.dumps(results, indent=2)
        
    except Exception as e:
        return f"Error executing query: {str(e)}"


def verify_sql_query(sql_query: str, schema_info: str, question: str) -> str:
    """
    Tool: Verify SQL query for correctness and safety
    """
    try:
        # Basic safety checks
        sql_upper = sql_query.strip().upper()
        
        # Check for dangerous operations
        dangerous_ops = ['DROP', 'DELETE', 'INSERT', 'UPDATE', 'ALTER', 'CREATE', 'TRUNCATE']
        for op in dangerous_ops:
            if op in sql_upper:
                return f"Error: Dangerous operation detected: {op}. Only SELECT queries are allowed."
        
        # Must start with SELECT
        if not sql_upper.startswith('SELECT'):
            return "Error: Only SELECT queries are allowed."
        
        # Check for basic SQL syntax issues
        if sql_query.count('(') != sql_query.count(')'):
            return "Error: Mismatched parentheses in SQL query."
        
        # Use Azure OpenAI to verify the query makes sense
        client = AzureOpenAI(
            azure_endpoint=AZURE_OPENAI_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
            api_version=AZURE_OPENAI_API_VERSION
        )
        
        prompt = f"""
Verify this SQL query for correctness and relevance to the question.

Database Schema:
{schema_info}

Question: {question}
SQL Query: {sql_query}

Check:
1. Does the query use valid table/column names from the schema?
2. Is the query logically correct for answering the question?
3. Are there any obvious syntax errors?

Return JSON with:
{{
    "is_valid": true/false,
    "issues": ["list of any issues found"],
    "suggestions": ["list of suggestions for improvement"]
}}
"""
        
        response = client.chat.completions.create(
            model=AZURE_OPENAI_MODEL,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            max_tokens=300,
            response_format={"type": "json_object"}
        )
        
        verification_result = json.loads(response.choices[0].message.content)
        
        if verification_result.get("is_valid", False):
            return "Query verified successfully."
        else:
            issues = verification_result.get("issues", ["Unknown validation error"])
            return f"Query validation failed: {', '.join(issues)}"
        
    except Exception as e:
        return f"Error verifying query: {str(e)}"


def format_response(question: str, sql_query: str, results: str) -> str:
    """
    Tool: Format results into a structured response object
    """
    try:
        client = AzureOpenAI(
            azure_endpoint=AZURE_OPENAI_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
            api_version=AZURE_OPENAI_API_VERSION
        )
        
        prompt = f"""
Convert these query results into a natural, conversational response.

Original Question: {question}
SQL Query: {sql_query}
Results: {results}

Provide a clear, helpful answer in natural language that explains the results.
"""
        
        response = client.chat.completions.create(
            model=AZURE_OPENAI_MODEL,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=400
        )
        
        natural_language_answer = response.choices[0].message.content.strip()
        
        # Parse the results for structured output
        import json
        try:
            parsed_results = json.loads(results)
        except:
            parsed_results = []
        
        # Return structured object as JSON string
        structured_response = {
            "response": natural_language_answer,
            "sql_query": sql_query,
            "data_results": parsed_results,
            "row_count": len(parsed_results) if isinstance(parsed_results, list) else 0
        }
        
        return json.dumps(structured_response, indent=2)
        
    except Exception as e:
        error_response = {
            "response": f"Error formatting response: {str(e)}",
            "sql_query": sql_query,
            "data_results": [],
            "row_count": 0
        }
        return json.dumps(error_response, indent=2)

print("✅ Tool functions defined")

✅ Tool functions defined


## 🤖 NL2SQL Agent

The main agent that orchestrates the entire process using function calling:

In [16]:
class NL2SQLAgent:
    """
    Simple NL2SQL Agent using Azure OpenAI function calling
    """
    
    def __init__(self):
        self.client = AzureOpenAI(
            azure_endpoint=AZURE_OPENAI_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
            api_version=AZURE_OPENAI_API_VERSION
        )
        
        # Define available tools
        self.tools = [
            {
                "type": "function",
                "function": {
                    "name": "discover_database_schema",
                    "description": "Discover database schema (tables and columns)",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "database_path": {"type": "string", "description": "Path to SQLite database"}
                        },
                        "required": ["database_path"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "generate_sql_query",
                    "description": "Generate SQL query from natural language",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "question": {"type": "string", "description": "Natural language question"},
                            "schema_info": {"type": "string", "description": "Database schema information"}
                        },
                        "required": ["question", "schema_info"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "verify_sql_query",
                    "description": "Verify SQL query for correctness and safety",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "sql_query": {"type": "string", "description": "SQL query to verify"},
                            "schema_info": {"type": "string", "description": "Database schema information"},
                            "question": {"type": "string", "description": "Original natural language question"}
                        },
                        "required": ["sql_query", "schema_info", "question"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "execute_sql_query",
                    "description": "Execute SQL query and get results",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "sql_query": {"type": "string", "description": "SQL query to execute"},
                            "database_path": {"type": "string", "description": "Path to SQLite database"}
                        },
                        "required": ["sql_query", "database_path"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "format_response",
                    "description": "Format results into natural language",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "question": {"type": "string", "description": "Original question"},
                            "sql_query": {"type": "string", "description": "SQL query used"},
                            "results": {"type": "string", "description": "Query results"}
                        },
                        "required": ["question", "sql_query", "results"]
                    }
                }
            }
        ]
        
        # Map function names to actual functions
        self.function_map = {
            "discover_database_schema": discover_database_schema,
            "generate_sql_query": generate_sql_query,
            "verify_sql_query": verify_sql_query,
            "execute_sql_query": execute_sql_query,
            "format_response": format_response
        }
    
    def process_question(self, question: str, database_path: str, show_logs: bool = True) -> str:
        """
        Process a natural language question using the agent with comprehensive logging
        """
        if show_logs:
            print(f"🚀 STARTING NL2SQL PROCESSING")
            print(f"📝 Question: {question}")
            print(f"🗄️ Database: {database_path}")
            print("=" * 80)
        
        messages = [
            {
                "role": "system",
                "content": """
You are an intelligent NL2SQL assistant that helps users query databases using natural language.

You have access to these tools:
- discover_database_schema: Learn about database structure
- generate_sql_query: Create SQL queries from natural language  
- verify_sql_query: Validate queries for safety and correctness
- execute_sql_query: Run queries against the database
- format_response: Convert results to natural language

You decide which tools to use and when. There's no fixed workflow - use your intelligence to:
1. Understand what information you need
2. Call appropriate tools to gather that information
3. Generate and validate SQL as needed
4. Execute queries safely
5. Provide helpful responses

Be intelligent about tool usage - you might need schema info before generating SQL, 
or you might want to verify a query before executing it. Use your judgment.

IMPORTANT: Always end by calling the format_response tool to provide the final structured response.
"""
            },
            {
                "role": "user", 
                "content": f"Database: {database_path}\nQuestion: {question}"
            }
        ]
        
        # Continue conversation until we have a final answer
        max_iterations = 15
        iteration = 0
        tool_call_count = 0
        final_structured_response = None
        
        while iteration < max_iterations:
            iteration += 1
            
            if show_logs:
                print(f"\n🔄 ITERATION {iteration}")
                print("-" * 40)
            
            response = self.client.chat.completions.create(
                model=AZURE_OPENAI_MODEL,
                messages=messages,
                tools=self.tools,
                tool_choice="auto",
                temperature=0.1
            )
            
            message = response.choices[0].message
            messages.append(message)
            
            # If no tool calls, we have the final answer
            if not message.tool_calls:
                if show_logs:
                    print(f"✅ FINAL RESPONSE GENERATED")
                    print(f"📊 Total iterations: {iteration}")
                    print(f"🛠️ Total tool calls: {tool_call_count}")
                    print("-" * 40)
                
                # Return the structured response if we have one, otherwise create a fallback
                if final_structured_response:
                    return final_structured_response
                else:
                    return {
                        "response": message.content,
                        "sql_query": "N/A",
                        "data_results": [],
                        "row_count": 0
                    }
            
            # Execute tool calls with detailed logging
            for tool_call in message.tool_calls:
                tool_call_count += 1
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                if show_logs:
                    print(f"🛠️ TOOL CALL #{tool_call_count}: {function_name}")
                    print(f"📥 Arguments: {function_args}")
                
                # Call the actual function
                import time
                start_time = time.time()
                result = self.function_map[function_name](**function_args)
                execution_time = (time.time() - start_time) * 1000
                
                if show_logs:
                    print(f"⏱️ Execution time: {execution_time:.2f}ms")
                    if len(str(result)) > 200:
                        print(f"📤 Result: {str(result)}")
                    else:
                        print(f"📤 Result: {result}")
                    print()
                
                # If this was the format_response call, capture the structured response
                if function_name == "format_response":
                    try:
                        final_structured_response = json.loads(result)
                    except:
                        final_structured_response = {
                            "response": result,
                            "sql_query": function_args.get("sql_query", "N/A"),
                            "data_results": [],
                            "row_count": 0
                        }
                
                # Add tool result to conversation
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": str(result)
                })
        
        # Fallback if we reach max iterations
        if final_structured_response:
            return final_structured_response
        else:
            return {
                "response": f"❌ Maximum iterations ({max_iterations}) reached. Unable to complete the request.",
                "sql_query": "N/A",
                "data_results": [],
                "row_count": 0
            }

print("✅ NL2SQL Agent defined")

✅ NL2SQL Agent defined


## 🧪 Demo and Testing

Let's test our simple agent with the Titanic dataset:

In [17]:
# Initialize the agent
agent = NL2SQLAgent()

# Database path
db_path = "../outputs/databases/titanic.db"

print("🤖 NL2SQL Agent initialized")
print(f"📊 Target database: {db_path}")
print("\n" + "="*60)
print("Ready for questions! 🚀")
print("="*60)

🤖 NL2SQL Agent initialized
📊 Target database: ../outputs/databases/titanic.db

Ready for questions! 🚀


## 🎯 Interactive Testing

Test the agent with your own questions:

In [18]:
def ask_question(question: str, database_path: str = "../outputs/databases/titanic.db", show_logs: bool = False):
    """
    Simple interface to ask questions - returns structured response
    """
    if not show_logs:
        print(f"🔍 Question: {question}")
        print("🤖 Processing...")
    
    try:
        result = agent.process_question(question, database_path, show_logs=show_logs)
        
        if not show_logs:
            print(f"\n💬 Response: {result['response']}")
            print(f"\n🔧 SQL Query:")
            print(f"```sql\n{result['sql_query']}\n```")
            print(f"\n📊 Data Results ({result['row_count']} rows):")
            
            # Display data in table format
            if result['data_results'] and len(result['data_results']) > 0:
                data = result['data_results']
                if len(data) > 0:
                    # Create table
                    columns = list(data[0].keys())
                    header = " | ".join(columns)
                    separator = " | ".join(["-" * len(col) for col in columns])
                    
                    print(f"```\n{header}\n{separator}")
                    for row in data[:10]:  # Show first 10 rows
                        row_values = [str(row.get(col, "")) for col in columns]
                        print(" | ".join(row_values))
                    if len(data) > 10:
                        print(f"... and {len(data) - 10} more rows")
                    print("```")
            else:
                print("```\nNo data returned\n```")
                
        else:
            print(f"\n💬 FINAL STRUCTURED RESULT:")
            print(f"Response: {result['response']}")
            print(f"SQL Query: {result['sql_query']}")
            print(f"Row Count: {result['row_count']}")
            print(f"Data Results: {result['data_results'][:3] if result['data_results'] else 'None'}{'...' if len(result.get('data_results', [])) > 3 else ''}")
        
        return result
        
    except Exception as e:
        error_result = {
            "response": f"Error: {str(e)}",
            "sql_query": "N/A",
            "data_results": [],
            "row_count": 0
        }
        if not show_logs:
            print(f"\n❌ Error: {str(e)}")
        return error_result

In [19]:
question = "considering those passengers who embarked at Queenstown but not from Southampton, and who might have been listed under the 2nd class but accidentally recorded as 3rd in some places, how many of them had siblings onboard, excluding those who were minors yet counted as adults in the family ticketing records?"

ask_question(question, db_path, show_logs=True)

🚀 STARTING NL2SQL PROCESSING
📝 Question: considering those passengers who embarked at Queenstown but not from Southampton, and who might have been listed under the 2nd class but accidentally recorded as 3rd in some places, how many of them had siblings onboard, excluding those who were minors yet counted as adults in the family ticketing records?
🗄️ Database: ../outputs/databases/titanic.db

🔄 ITERATION 1
----------------------------------------


🛠️ TOOL CALL #1: discover_database_schema
📥 Arguments: {'database_path': '../outputs/databases/titanic.db'}
⏱️ Execution time: 48.35ms
📤 Result: {
  "titanic": [
    {
      "name": "PassengerId",
      "type": "INTEGER",
      "nullable": true
    },
    {
      "name": "Survived",
      "type": "INTEGER",
      "nullable": true
    },
    {
      "name": "Pclass",
      "type": "INTEGER",
      "nullable": true
    },
    {
      "name": "Name",
      "type": "TEXT",
      "nullable": true
    },
    {
      "name": "Sex",
      "type": "TEXT",
      "nullable": true
    },
    {
      "name": "Age",
      "type": "REAL",
      "nullable": true
    },
    {
      "name": "SibSp",
      "type": "INTEGER",
      "nullable": true
    },
    {
      "name": "Parch",
      "type": "INTEGER",
      "nullable": true
    },
    {
      "name": "Ticket",
      "type": "TEXT",
      "nullable": true
    },
    {
      "name": "Fare",
      "type": "REAL",
      "nullable": true
    },
    {
  

{'response': "Error: 'int' object is not subscriptable",
 'sql_query': 'N/A',
 'data_results': [],
 'row_count': 0}

In [20]:
question = "How many female passengers named Mary or Maria, excluding those who had cabins starting with letter 'C', survived if they also belonged to families where the ticket was shared by more than three relatives?"

ask_question(question, db_path, show_logs=True)

🚀 STARTING NL2SQL PROCESSING
📝 Question: How many female passengers named Mary or Maria, excluding those who had cabins starting with letter 'C', survived if they also belonged to families where the ticket was shared by more than three relatives?
🗄️ Database: ../outputs/databases/titanic.db

🔄 ITERATION 1
----------------------------------------
🛠️ TOOL CALL #1: discover_database_schema
📥 Arguments: {'database_path': '../outputs/databases/titanic.db'}
⏱️ Execution time: 57.30ms
📤 Result: {
  "titanic": [
    {
      "name": "PassengerId",
      "type": "INTEGER",
      "nullable": true
    },
    {
      "name": "Survived",
      "type": "INTEGER",
      "nullable": true
    },
    {
      "name": "Pclass",
      "type": "INTEGER",
      "nullable": true
    },
    {
      "name": "Name",
      "type": "TEXT",
      "nullable": true
    },
    {
      "name": "Sex",
      "type": "TEXT",
      "nullable": true
    },
    {
      "name": "Age",
      "type": "REAL",
      "nullable": tr

{'response': "Based on the information you provided, there were no female passengers named Mary or Maria—who did not have a cabin starting with the letter 'C', survived, and were traveling with more than three relatives on the same ticket—on the Titanic. In other words, no one met all of these specific criteria.",
 'sql_query': "SELECT COUNT(*) as count\nFROM titanic\nWHERE Sex = 'female'\n  AND Survived = 1\n  AND (Name LIKE '% Mary %' OR Name LIKE '% Maria %')\n  AND (Cabin IS NULL OR Cabin NOT LIKE 'C%')\n  AND (SibSp + Parch) > 0\n  AND Ticket IN (\n      SELECT Ticket\n      FROM titanic\n      WHERE (SibSp + Parch) > 0\n      GROUP BY Ticket\n      HAVING COUNT(*) > 3\n  );",
 'data_results': [{'count': 0}],
 'row_count': 1}

In [21]:
question = "What is the average fare paid by passengers who embarked from Cherbourg and Southampton at the same time, even though it was not possible, but assuming a data inconsistency could have logged them as both?"

ask_question(question, db_path, show_logs=True)

🚀 STARTING NL2SQL PROCESSING
📝 Question: What is the average fare paid by passengers who embarked from Cherbourg and Southampton at the same time, even though it was not possible, but assuming a data inconsistency could have logged them as both?
🗄️ Database: ../outputs/databases/titanic.db

🔄 ITERATION 1
----------------------------------------
🛠️ TOOL CALL #1: discover_database_schema
📥 Arguments: {'database_path': '../outputs/databases/titanic.db'}
⏱️ Execution time: 89.37ms
📤 Result: {
  "titanic": [
    {
      "name": "PassengerId",
      "type": "INTEGER",
      "nullable": true
    },
    {
      "name": "Survived",
      "type": "INTEGER",
      "nullable": true
    },
    {
      "name": "Pclass",
      "type": "INTEGER",
      "nullable": true
    },
    {
      "name": "Name",
      "type": "TEXT",
      "nullable": true
    },
    {
      "name": "Sex",
      "type": "TEXT",
      "nullable": true
    },
    {
      "name": "Age",
      "type": "REAL",
      "nullable": tru

{'response': 'It looks like there were no passengers in the data who were recorded as having embarked from both Cherbourg and Southampton at the same time. As a result, there isn’t an average fare to report for this scenario—the database returned no value because such a case doesn’t exist in the records, even if it might have been possible due to a data inconsistency.',
 'sql_query': "SELECT AVG(Fare) AS average_fare FROM titanic WHERE Embarked LIKE '%C%' AND Embarked LIKE '%S%';",
 'data_results': [{'average_fare': None}],
 'row_count': 1}

In [22]:
question = "Among those passengers who died but had been incorrectly marked as survived in secondary versions of the dataset, what percentage had a recorded age greater than 70 but later corrected to below 60?"

ask_question(question, db_path, show_logs=True)

🚀 STARTING NL2SQL PROCESSING
📝 Question: Among those passengers who died but had been incorrectly marked as survived in secondary versions of the dataset, what percentage had a recorded age greater than 70 but later corrected to below 60?
🗄️ Database: ../outputs/databases/titanic.db

🔄 ITERATION 1
----------------------------------------
🛠️ TOOL CALL #1: discover_database_schema
📥 Arguments: {'database_path': '../outputs/databases/titanic.db'}
⏱️ Execution time: 89.34ms
📤 Result: {
  "titanic": [
    {
      "name": "PassengerId",
      "type": "INTEGER",
      "nullable": true
    },
    {
      "name": "Survived",
      "type": "INTEGER",
      "nullable": true
    },
    {
      "name": "Pclass",
      "type": "INTEGER",
      "nullable": true
    },
    {
      "name": "Name",
      "type": "TEXT",
      "nullable": true
    },
    {
      "name": "Sex",
      "type": "TEXT",
      "nullable": true
    },
    {
      "name": "Age",
      "type": "REAL",
      "nullable": true
    }

{'response': "Unfortunately, we can't answer this question with the current data available. The Titanic dataset we have only contains a single, current record for each passenger, without any history of previous or corrected values. Because of this, we can't identify which passengers were ever incorrectly marked as survived, nor can we track changes in their recorded ages over time. To answer your question, we would need a versioned dataset that shows how each passenger's information may have changed, including both the original and corrected values for survival status and age.",
 'sql_query': 'The question cannot be answered with the current database schema, as there is no reference to a secondary version of the dataset or a way to compare changes in Survived or Age values over time.',
 'data_results': [],
 'row_count': 0}

In [23]:
question = "If we only consider children younger than 5 years who were classified as 'Master' in the name field but exclude those tagged with 'Miss', how many survived?"

ask_question(question, db_path, show_logs=True)

🚀 STARTING NL2SQL PROCESSING
📝 Question: If we only consider children younger than 5 years who were classified as 'Master' in the name field but exclude those tagged with 'Miss', how many survived?
🗄️ Database: ../outputs/databases/titanic.db

🔄 ITERATION 1
----------------------------------------
🛠️ TOOL CALL #1: discover_database_schema
📥 Arguments: {'database_path': '../outputs/databases/titanic.db'}
⏱️ Execution time: 82.24ms
📤 Result: {
  "titanic": [
    {
      "name": "PassengerId",
      "type": "INTEGER",
      "nullable": true
    },
    {
      "name": "Survived",
      "type": "INTEGER",
      "nullable": true
    },
    {
      "name": "Pclass",
      "type": "INTEGER",
      "nullable": true
    },
    {
      "name": "Name",
      "type": "TEXT",
      "nullable": true
    },
    {
      "name": "Sex",
      "type": "TEXT",
      "nullable": true
    },
    {
      "name": "Age",
      "type": "REAL",
      "nullable": true
    },
    {
      "name": "SibSp",
      "typ

{'response': "Certainly! Based on your criteria—looking only at children under 5 years old who were listed as 'Master' in their name (and not 'Miss')—there were 15 boys in this group who survived the Titanic disaster. These children were all boys, as 'Master' was the title used for young boys at the time. Their ages ranged from just a few months old up to 4 years old. Some examples include Master Alden Gates Caldwell (less than a year old), Master Michel M Navratil (3 years old), and Master Harold Theodor Johnson (4 years old). So, to summarize, 15 boys under the age of 5, identified as 'Master' (and not 'Miss'), survived.",
 'sql_query': "SELECT PassengerId, Name, Age, Survived\nFROM titanic\nWHERE Age < 5\n  AND Name LIKE '%Master%'\n  AND Name NOT LIKE '%Miss%'\n  AND Survived = 1;",
 'data_results': [],
 'row_count': 0}

In [24]:
question = "What is the total number of passengers with cabin numbers starting with 'B' who embarked in Queenstown and also had more than 2 parents/children onboard, provided they also had no recorded ticket number?"

ask_question(question, db_path, show_logs=True)

🚀 STARTING NL2SQL PROCESSING
📝 Question: What is the total number of passengers with cabin numbers starting with 'B' who embarked in Queenstown and also had more than 2 parents/children onboard, provided they also had no recorded ticket number?
🗄️ Database: ../outputs/databases/titanic.db

🔄 ITERATION 1
----------------------------------------
🛠️ TOOL CALL #1: discover_database_schema
📥 Arguments: {'database_path': '../outputs/databases/titanic.db'}
⏱️ Execution time: 80.01ms
📤 Result: {
  "titanic": [
    {
      "name": "PassengerId",
      "type": "INTEGER",
      "nullable": true
    },
    {
      "name": "Survived",
      "type": "INTEGER",
      "nullable": true
    },
    {
      "name": "Pclass",
      "type": "INTEGER",
      "nullable": true
    },
    {
      "name": "Name",
      "type": "TEXT",
      "nullable": true
    },
    {
      "name": "Sex",
      "type": "TEXT",
      "nullable": true
    },
    {
      "name": "Age",
      "type": "REAL",
      "nullable": true

{'response': "There were no passengers who met all of those criteria. In other words, there weren’t any passengers with a cabin number starting with 'B', who embarked in Queenstown, had more than two parents or children onboard, and also had no recorded ticket number.",
 'sql_query': "SELECT *\nFROM titanic\nWHERE Cabin LIKE 'B%'\n  AND Embarked = 'Q'\n  AND Parch > 2\n  AND (Ticket IS NULL OR TRIM(Ticket) = '');",
 'data_results': [],
 'row_count': 0}